# Install Ultralytics for YOLOv8 model

In [ ]:
import os
import subprocess
import boto3, sagemaker
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorchModel
from sagemaker.deserializers import JSONDeserializer

In [ ]:
!pip3 install ultralytics

In [ ]:
from ultralytics import YOLO

modelName = 'yolov8x.pt'

model = YOLO(modelName)

# Save the model as a pytorch file
model.save('yolov8x.pt')

# Zip the code and model into `model.tar.gz`

In [18]:
tarFile = 'model.tar.gz'
helperFileDir = 'code/'

bashCommand = f"tar -cpzf  {tarFile} {modelName} {helperFileDir}"
process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
output, error = process.communicate()

# Upload the model to an S3 bucket

In [ ]:
# Get the default S3 bucket name

sess = sagemaker.Session()
bucket = sess.default_bucket()
print(bucket) # the default S3 bucket allocated for this SageMaker session

In [ ]:
# Upload the model to S3

s3 = boto3.client('s3')
s3.upload_file(
    Filename=tarFile,
    Bucket=bucket,
    Key=os.path.basename(tarFile)
)

modelUri = f"s3://{bucket}/{tarFile}"
print(modelUri)

# Create the SageMaker PyTorchModel

In [21]:
entryPoint = 'inference.py'
roleArn = get_execution_role()

In [22]:
model = PyTorchModel(entry_point=entryPoint,
                     model_data=modelUri, 
                     framework_version='1.12', 
                     py_version='py38',
                     role=roleArn,
                     env={'TS_MAX_RESPONSE_SIZE':'20000000', 'YOLOV8_MODEL': modelName},
                     sagemaker_session=sess)

# Deploy the model on SageMaker Endpoint

In [23]:
endpointName = "YOLOv8"
instanceType = "ml.m5.xlarge"

In [ ]:
predictor = model.deploy(initial_instance_count=1, 
                         instance_type=instanceType,
                         deserializer=JSONDeserializer(),
                         endpoint_name=endpointName)

# Check if Endpoint creation is successful and create the predictor

In [ ]:
import boto3, time

In [ ]:
sm_client = boto3.client(service_name="sagemaker")

endpoint_created = False
while True:
    response = sm_client.list_endpoints()
    for ep in response['Endpoints']:
        print(f"Endpoint Status = {ep['EndpointStatus']}")
        if ep['EndpointName']==endpointName and ep['EndpointStatus']=='InService':
            endpoint_created = True
            break
    if endpoint_created:
        break
    time.sleep(5)

# Cleanup by removing Endpoint, Endpoint Config and Model

In [ ]:
response = sm_client.describe_endpoint_config(EndpointConfigName=endpointName)
print(response)
endpoint_config_name = response['EndpointConfigName']

# Delete Endpoint
sm_client.delete_endpoint(EndpointName=endpointName)

# Delete Endpoint Configuration
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)

# Delete Model
for prod_var in response['ProductionVariants']:
    model_name = prod_var['ModelName']
    sm_client.delete_model(ModelName=model_name)     